In [16]:
# ORIGINAL CODE : https://machinelearningmastery.com/weighted-average-ensemble-with-python/
# By Jason Brownlee - MAY 2021

# Modifed by MAHAYASA ADIPUTRA - OCT 2022
import time
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix ,accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from sklearn.preprocessing import LabelEncoder
import sklearn.metrics as mt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import VotingClassifier
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import NeighbourhoodCleaningRule
le = LabelEncoder()
start_time = time.time()
# get the dataset
# Importing the dataset
dataset=pd.read_csv('https://raw.githubusercontent.com/mahayasa/various-sampling-churn-prediction/main/data/telco-non-internet.csv')
dataset=dataset.dropna()
#dataset=dataset.fillna(0)

dataset['state']=le.fit_transform(dataset['state'])
dataset['area_code']=le.fit_transform(dataset['area_code'])
dataset['international_plan']=le.fit_transform(dataset['international_plan'])
dataset['voice_mail_plan']=le.fit_transform(dataset['voice_mail_plan'])
dataset['churn']=le.fit_transform(dataset['churn'])

#filling missing value with mean
#dataset['TotalCharges'].replace(to_replace = 0, value = dataset['TotalCharges'].mean(), inplace=True)
X=dataset.drop(['churn'],axis=1)
y=dataset["churn"]
#imblearn SMOTE
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)
#smote_enn = SMOTEENN()
#smote_enn = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
#X, y = smote_enn.fit_resample(X, y)
ncr = NeighbourhoodCleaningRule(n_neighbors=5, kind_sel='all')
X, y = ncr.fit_resample(X, y)

#kfold cross validation
cv = KFold(n_splits=5, random_state=1, shuffle=True)
# get a list of base models
def get_models():
	models = list()
	models.append(('xb', XGBClassifier(colsample_bytree=0.6, max_depth = 5, gamma=1.5, min_child_weight=5)))
	models.append(('rf', RandomForestClassifier(n_estimators =100, criterion = 'gini', random_state = 0)))
#	models.append(('ab', AdaBoostClassifier(n_estimators=100,learning_rate=0.1,algorithm='SAMME.R')))
	return models
 
# evaluate each base model
def evaluate_models(models, X,y):
	# fit and evaluate the models
	scores = list()
	for name, model in models:
		# evaluate the model
		acc=cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
		# store the performance
		scores.append(mean(acc))
		# report model performance
	return scores



# create the base models
models = get_models()
# fit and evaluate each model
scores = evaluate_models(models, X,y)
print(scores)
# create the ensemble
ensemble = VotingClassifier(estimators=models, voting='soft', weights=scores)
score=cross_val_score(ensemble, X, y, scoring='f1', cv=cv, n_jobs=-1)
print('F1 score: %.3f' % (mean(score)))
print('STD F1 Score: %.3f' % (std(score)))

[0.9753191361300791, 0.9844218164775811]
F1 score: 0.932
STD F1 Score: 0.002


In [17]:
rc=cross_val_score(ensemble, X, y, scoring='recall', cv=cv, n_jobs=-1)
rc=mean(rc)
rc

0.9058230444574402

In [18]:
from sklearn.metrics import make_scorer
from imblearn.metrics import specificity_score

# Define the specificity scorer
scorer = make_scorer(specificity_score)

# Calculate the cross-validated specificity score
sp = cross_val_score(ensemble, X, y, cv=5, scoring=scorer)
sp=mean(sp)
sp

0.9602968348855949

In [19]:
auc=cross_val_score(ensemble, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
auc=mean(auc)
auc

0.9830859762050116

In [20]:
import math

gmean=rc*sp
sqrtg = math.sqrt(gmean)
sqrtg

0.9326623196843075